In [1]:
import numpy as np
import cv2
import os
#from pygame import mixer # Load the required library
import requests
import time


In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.13.1


In [4]:
from keras.preprocessing.image import array_to_img, img_to_array
from keras.models import model_from_json

Using TensorFlow backend.


In [5]:
def read_model(network_path):
    exit_ifnex(network_path)
    model = model_from_json(open(os.path.join(network_path, 'architecture.json')).read())
    model.load_weights(os.path.join(network_path, 'weights.h5'))
    return model

In [6]:
def max_index_of(array):
    m = -1
    index = -1
    for i in range(len(array)):
        if array[i] > m:
            m = array[i]
            index = i
    return index

In [7]:
def exit_ifnex(directory):
    if not os.path.exists(directory):
        print(directory, 'does not exist')
        exit()
def timestamp():
    return int(round(time.time() * 1000))

In [8]:
# load neural network
model = read_model('../model2')
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
width, height, channel = 300, 280, 1

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
background = None

# Start with a halfway point between 0 and 1 of accumulated weight
accumulated_weight = 0.5


# Manually set up our ROI for grabbing the hand.
# Feel free to change these. I just chose the top right corner for filming.
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [10]:
def calc_accum_avg(frame, accumulated_weight):
    '''
    Given a frame and a previous accumulated weight, computed the weighted average of the image passed in.
    '''
    
    # Grab the background
    global background
    
    # For first time, create the background from a copy of the frame.
    if background is None:
        background = frame.copy().astype("float")
        return None

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [11]:
def segment(frame, threshold_min=25):
    global background
    
    # Calculates the Absolute Differentce between the backgroud and the passed in frame
    diff = cv2.absdiff(background.astype("uint8"), frame)

    # Apply a threshold to the image so we can grab the foreground
    # We only need the threshold, so we will throw away the first item in the tuple with an underscore _
    ret , thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)

    # Grab the external contours form the image
    # Again, only grabbing what we need here and throwing away the rest
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If length of contours list is 0, then we didn't grab any contours!
    if len(contours) == 0:
        return None
    else:
        # Given the way we are using the program, the largest external contour should be the hand (largest by area)
        # This will be our segment
        hand_segment = max(contours, key=cv2.contourArea)
        
        # Return both the hand segment and the thresholded hand image
        return (thresholded, hand_segment)

In [12]:
from time import sleep
from threading import *
from pygame import mixer # Load the required library
k=0
url="https://translate.google.com/translate_tts"
text=""

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
class play(Thread):
    def run(self):
        global text
        global k
        while True:
            if len(text)>10:
                params={'ie':'UTF-8','q': text,'tl': 'en','client': 'gtx'}
                r= requests.get(url,params=params,headers=headers)
                
                with open(f"thread{k}.mp3",'wb') as f:
                    f.write(r.content)
                mixer.init()
                mixer.music.load(f"thread{k}.mp3")
                mixer.music.play()
                print(k)
                
                if k==0:
                    if os.path.exists("thread1.mp3"):
                        os.remove("thread1.mp3")
                    k=1
                else:
                    if os.path.exists("thread0.mp3"):
                        os.remove("thread0.mp3")
                    k=0
                #print(text)
                text=""
                

In [16]:
#thread=play()
#thread.start()

previous=0
cam = cv2.VideoCapture(0)
# capture settings
time_between_capture_ms = 1500
last_capture = timestamp()

# Intialize a frame count
num_frames = 0

while True:
    # get the current frame
    ret, frame = cam.read()

    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)

    # clone the frame
    frame_copy = frame.copy()

    # Grab the ROI from the frame
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # Apply grayscale and blur to ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # For the first 30 frames we will calculate the average of the background.
    # We will tell the user while this is happening
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        # now that we have the background, we can segment the hand.
        
        # segment the hand region
        hand = segment(gray)

        # First check if we were able to actually detect a hand
        if hand is not None:
            
            # unpack
            thresholded, hand_segment = hand

            # Draw contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

            # Count the fingers
            #fingers = count_fingers(thresholded, hand_segment)
            
            # Display count
            #cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # Also display the thresholded image
            cv2.imshow("Thesholded", thresholded)
            image = cv2.resize(thresholded, (width, height))
            image = img_to_array(image)
            image = np.array(image, dtype="float") / 255.0
            image = image.reshape(1, width, height, channel)
            if timestamp()-last_capture>time_between_capture_ms:
                output = model.predict(image)
                os.system('clear')
                
                if max_index_of(output[0])==0 and previous != 0:
                    previous=0
                    mixer.init()
                    mixer.music.load("fine.mp3")
                    mixer.music.play()
                    text=text+" I" #paper
                if max_index_of(output[0])==1 and previous != 1:
                    previous=1
                    mixer.init()
                    mixer.music.load("how.mp3")
                    mixer.music.play()
                if max_index_of(output[0])==2 and previous != 2:
                    previous=2
                    mixer.init()
                    mixer.music.load("you.mp3")
                    mixer.music.play()
                if max_index_of(output[0])==3 and previous != 3:
                    previous=3
                    mixer.init()
                    mixer.music.load("I.mp3")
                    mixer.music.play()
                    
                if max_index_of(output[0])==4 and previous != 4:
                    previous=4
                    mixer.init()
                    mixer.music.load("are.mp3")
                    mixer.music.play()
                if max_index_of(output[0])==5 and previous != 5:
                    previous=5
                    mixer.init()
                    mixer.music.load("hi.mp3")
                    mixer.music.play()
                
                if max_index_of(output[0])==7 and previous != 7:
                    previous=7
                    mixer.init()
                    mixer.music.load("am.mp3")
                    mixer.music.play()
                #print(max_index_of(output[0]))   
                last_capture = timestamp()
            
    # Draw ROI Rectangle on frame copy
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    # increment the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()
